# Assignment: Introduction to RAG Systems
---

Welcome to the first assignment in the RAG course! In this assignment, you'll build a RAG pipeline using a dataset containing news information from BBC News. The goal is to enable the LLM to retrieve relevant news details from the dataset and use that information to generate more informed responses. The model you'll be using is the [llama-3-1-8b-instruct-turbo](https://www.together.ai/models/llama-3-1), which was trained on data up to December 2023. The idea is to create a RAG system that allows it to include information on events that occurred in 2024.

In this assignment, you will:
- Use a query and retrieval function to access relevant data given a query
- Format the data appropriately
- Generate a prompt with the query and the relevant data to feed into the LLM


# Table of Contents
- [ 1 - Introduction](#1)
  - [ 1.1 RAG architecture overview](#1-1)
  - [ 1.2 Importing the necessary libraries](#1-2)
- [ 2 - Loading the dataset](#2)
- [ 3 - Main Functions](#3)
  - [ 3.1 Query news by index function](#3-1)
  - [ 3.2 Retrieve function](#3-2)
  - [ 3.3 Get relevant data](#3-3)
    - [ Exercise 1](#ex01)
  - [ 3.4 Formatting the relevant rata](#3-4)
    - [ Exercise 2](#ex02)
  - [ 3.5 Generate the final prompt](#3-5)
  - [ 3.6 LLM call](#3-6)
- [ 4 - Experimenting with your RAG System](#4)


---
<h4 style="color:black; font-weight:bold;">USING THE TABLE OF CONTENTS</h4>

JupyterLab provides an easy way for you to navigate through your assignment. It's located under the Table of Contents tab, found in the left panel, as shown in the picture below.

![TOC Location](images/toc.png)

---

<h4 style="color:green; font-weight:bold;">TIPS FOR SUCCESSFUL GRADING OF YOUR ASSIGNMENT:</h4>

- All cells are frozen except for the ones where you need to submit your solutions or when explicitly mentioned you can interact with it.

- You can add new cells to experiment but these will be omitted by the grader, so don't rely on newly created cells to host your solution code, use the provided places for this.

- Avoid using global variables unless you absolutely have to. The grader tests your code in an isolated environment without running all cells from the top. As a result, global variables may be unavailable when scoring your submission. Global variables that are meant to be used will be defined in UPPERCASE.

- - To submit your notebook for grading, first save it by clicking the 💾 icon on the top left of the page and then click on the <span style="background-color: blue; color: white; padding: 3px 5px; font-size: 16px; border-radius: 5px;">Submit assignment</span> button on the top right of the page.
---

<a id='1'></a>
## 1 - Introduction

---

<a id='1-1'></a>
### 1.1 RAG architecture overview

As presented in the lectures, a simplified diagram for RAG is as follows:

<div align="center">
  <img src="images/rag_overview.png" alt="RAG Overview" width="60%">
</div>

This assignment is designed to guide you through the general workflow. You will employ a pre-implemented retriever to obtain relevant data for a given query, format this data, and create a new prompt that includes both the query and the retrieved information. Finally, you will have the opportunity to compare the results of queries both with and without the RAG system to assess its impact on the LLM's response. 

Alright, let's roll up our sleeves and get started!

<a id='1-2'></a>
### 1.2 Importing the necessary libraries

Run the cell below to import the necessary libraries for this assignment.

In [1]:
from utils import (
    retrieve, 
    pprint, 
    generate_with_single_input, 
    read_dataframe, 
    display_widget
)
import unittests

<a id='2'></a>

<a id='2'></a>
## 2 - Loading the dataset

---

In this assignment, you’ll work with the Kaggle dataset [News Headlines 2024](https://www.kaggle.com/datasets/dylanjcastillo/news-headlines-2024). This dataset contains thousands of news headlines and related information from BBC News.


In [2]:
NEWS_DATA = read_dataframe("news_data_dedup.csv")

Let's check the data structure.

In [3]:
pprint(NEWS_DATA[9:11])

[
  {
    "guid": "5dae28f191cfd1047f67c409e616fc3f",
    "title": "Paris's Moulin Rouge loses windmill sails overnight",
    "description": "The cause of the sails' collapse from the roof of the world famous cabaret club is not yet clear.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-europe-68895836",
    "published_at": "2024-04-25",
    "updated_at": "2024-04-26"
  },
  {
    "guid": "d2c3ff79d4e068911d05416ca061cd51",
    "title": "Ukraine uses longer-range US missiles for first time",
    "description": "Missiles secretly delivered this month have been used to strike Russian targets in Crimea, US media say.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-europe-68893196",
    "published_at": "2024-04-25",
    "updated_at": "2024-04-26"
  }
]


Important fields are `title`, `description`, `url` and `published_at`. These fields will give good information to the LLM to answer the majority of questions with good enough data.

<a id='3'></a>
## 3 - Main Functions
---
Two functions will be provided to you:
- `query_news`: Given a list of indices, this function returns all documents corresponding to those indices.
- `retrieve`: Given a query and an integer called `top_k`, this function retrieves the `top_k` most relevant documents.

The functions you will implement are:
- `get_relevant_data`: This function takes a query and a `top_k` value and returns the `top_k` relevant documents.
- `format_relevant_data`: Given a list of documents, this function creates a formatted string with the document information.

You will then use these functions to create your own small RAG pipeline and see it in action!

<a id='3-1'></a>
### 3.1 Query news by index function

This simple function just simplifies the return of documents given a list of indices. It is given to you.

In [4]:
def query_news(indices):
    """
    Retrieves elements from a dataset based on specified indices.

    Parameters:
    indices (list of int): A list containing the indices of the desired elements in the dataset.
    dataset (list or sequence): The dataset from which elements are to be retrieved. It should support indexing.

    Returns:
    list: A list of elements from the dataset corresponding to the indices provided in list_of_indices.
    """
     
    output = [NEWS_DATA[index] for index in indices]

    return output

In [5]:
# Fetching some indices
indices = [3, 6, 9]
pprint(query_news(indices))

[
  {
    "guid": "e696224ac208878a5cec8bdc9f97c632",
    "title": "Europe risks dying and faces big decisions - Macron",
    "description": "The French president delivers a stark warning for Europe to act fast to survive in a changing world.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-europe-68898887",
    "published_at": "2024-04-25",
    "updated_at": "2024-04-26"
  },
  {
    "guid": "4f585bad8f61b715fbafe2f022ab0ae8",
    "title": "Supreme Court divided on whether Trump has immunity",
    "description": "The justices discussed immunity, coups, pardons, Operation Mongoose - and the future of democracy.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-us-canada-68901817",
    "published_at": "2024-04-25",
    "updated_at": "2024-04-26"
  },
  {
    "guid": "5dae28f191cfd1047f67c409e616fc3f",
    "title": "Paris's Moulin Rouge loses windmill sails overnight",
    "description": "The cause of the sails' collapse from the roof of the world famou

<a id='3-2'></a>
### 3.2 Retrieve function

The `retrieve` function is an essential part of our RAG system. While the full code is provided in the `utils.py` file, and you can examine it there, you will focus on understanding its input parameters and output for now. In Module 2, you will explore the detailed workings and various techniques for document retrieval.

**Parameters:**
- `query`: A string representing the search query for which you want to find the most relevant documents.
- `top_k`: An integer indicating the number of top similar documents to return.

**Output:**
- The function returns a list of indices corresponding to the top `k` most similar documents from the corpus, based on their similarity scores with the query.

**Call:**

```Python
retrieve(query: str, top_k: int)
```

As you move forward in this course, you'll gain deeper insights into how this function utilizes embeddings and similarity measures to perform effective document retrieval.

In [6]:
# Let's test the retrieve function!
indices = retrieve("Concerts in North America", top_k = 1)
print(indices)

[350]


In [7]:
# Now let's query the corresponding news_
retrieved_documents = query_news(indices)
pprint(retrieved_documents)

[
  {
    "guid": "927257674585bb6ef669cf2c2f409fa7",
    "title": "\u2018The working class can\u2019t afford it\u2019: the shocking truth about the money bands make on tour",
    "description": "As Taylor Swift tops $1bn in tour revenue, musicians playing smaller venues are facing pitiful fees and frequent losses. Should the state step in to save our live music scene?When you see a band playing to thousands of fans in a sun-drenched festival field, signing a record deal with a major label or playing endlessly from the airwaves, it\u2019s easy to conjure an image of success that comes with some serious cash to boot \u2013 particularly when Taylor Swift has broken $1bn in revenue for her current Eras tour. But looks can be deceiving. \u201cI don\u2019t blame the public for seeing a band playing to 2,000 people and thinking they\u2019re minted,\u201d says artist manager Dan Potts. \u201cBut the reality is quite different.\u201dPost-Covid there has been significant focus on grassroots mus

<a id='3-3'></a>
### 3.3 Get relevant data

<a id='ex01'></a>
### Exercise 1

In this exercise, you will create a function that inputs a `query` a `top_k` and returns a list with the `relevant items`. You will consolidate the two functions described so far into only one.

<details>
<summary style="color: green;">Hint 1</summary>
The function to get the relevant indices is the <code>retrieve</code> function. Remember that it inputs a <code>query</code> and the <code>top_k</code> and returns a <b>list of indices</b>.
</details>
<details>
<summary style="color: green;">Hint 2</summary>
The function to get the relevant data is the <code>query_news</code> function. It inputs a <b>set of indices</b> and outputs a list with the relevant data.
</details>

In [9]:
# GRADED CELL 

def get_relevant_data(query: str, top_k: int = 5) -> list[dict]:
    """
    Retrieve and return the top relevant data items based on a given query.

    This function performs the following steps:
    1. Retrieves the indices of the top 'k' relevant items from a dataset based on the provided `query`.
    2. Fetches the corresponding data for these indices from the dataset.

    Parameters:
    - query (str): The search query string used to find relevant items.
    - top_k (int, optional): The number of top items to retrieve. Default is 5.

    Returns:
    - list[dict]: A list of dictionaries containing the data associated 
      with the top relevant items.

    """
    ### START CODE HERE ###

    # Retrieve the indices of the top_k relevant items given the query
    relevant_indices = retrieve(query, top_k = top_k)

    # Obtain the data related to the items using the indices from the previous step
    relevant_data = query_news(relevant_indices)

    ### END CODE HERE
    
    return relevant_data

In [10]:
query = "Greatest storms in the US"
relevant_data = get_relevant_data(query, top_k = 1)
pprint(relevant_data)

[
  {
    "guid": "3ca548fe82c3fcae2c4c0c635d03eb2e",
    "title": "Large tornado seen touching down in Nebraska",
    "description": "Severe and powerful storms have moved across several US states, leaving many experiencing power shortages.",
    "venue": "BBC",
    "url": "https://www.bbc.co.uk/news/world-us-canada-68860070",
    "published_at": "2024-04-26",
    "updated_at": "2024-04-28"
  }
]


**Expected output**
```
[{'guid': '3ca548fe82c3fcae2c4c0c635d03eb2e',
  'title': 'Large tornado seen touching down in Nebraska',
  'description': 'Severe and powerful storms have moved across several US '
                 'states, leaving many experiencing power shortages.',
  'venue': 'BBC',
  'url': 'https://www.bbc.co.uk/news/world-us-canada-68860070',
  'published_at': '2024-04-26',
  'updated_at': '2024-04-28'}]
```

In [11]:
# Run this cell to perform several tests on your function. If you receive "All test passed!" it means that your solution will likely pass the autograder too.
unittests.test_get_relevant_data(get_relevant_data)

 All tests passed!


<a id='3-4'></a>

<a id='3-4'></a>
### 3.4 Formatting the relevant rata


<a id='ex02'></a>

<a id='ex02'></a>
### Exercise 2

In this exercise, you will create the `format_relevant_data` function that takes a list of data and formats it.

**Note:** You can adjust the layout, but your output **must** include these pieces of information:

* News title
* News description
* News published date
* News URL

You must include the exact keywords `'title'`, `'url'`, `'published'`, and `'description'` somewhere in your output (uppercase or lowercase versions are fine). These are required for grading.

**Tip:** It’s recommended to use **double quotes** for strings and **single quotes** for dictionary keys (for example, `data['title']`).
Here’s one way you could format it:

```python
f"""
Title: {news_1_title}, Description: {news_1_description}, Published at: {news_1_published_date}\nURL: {news_1_URL}
Title: {news_2_title}, Description: {news_2_description}, Published at: {news_2_published_date}\nURL: {news_2_URL}
...
Title: {news_k_title}, Description: {news_k_description}, Published at: {news_k_published_date}\nURL: {news_k_URL}
"""
```

<details>  
<summary style="color: green;">Hint 1</summary>  
<p>You can access each property of a document using its key in the dictionary.</p>  
<p>For example, to get the title: <code>document['title']</code></p>  
<p>You can format it like this: <code>f"Title: {document['title']}"</code></p>  
</details>

<details>  
<summary style="color: green;">Hint 2</summary>  
<p>Remember to append each formatted document to the <code>formatted_documents</code> list by doing <code>formatted_documents.append(formatted_document)</code>.</p>  
</details>

In [34]:
# GRADED CELL

def format_relevant_data(relevant_data):
    """
    Retrieves the top_k most relevant documents based on a given query and constructs an augmented prompt for a RAG system.

    Parameters:
    relevant_data (list): A list with relevant data.

    Returns:
    str: An augmented prompt with the top_k relevant documents, formatted for use in a Retrieval-Augmented Generation (RAG) system."
    """

    ### START CODE HERE ###

    # Create a list so store the formatted documents
    formatted_documents = []
    
    # Iterates over each relevant document.
    for document in relevant_data:

        # Formats each document into a structured layout string. Remember that each document is in one different line. So you should add a new line character after each document added.
        formatted_document = (
            f"Title: {document['title']}, "
            f"Description: {document['description']}, "
            f"Published at: {document['published_at']}, "
            f"URL: {document['url']},"
        )
        # Append the formatted document string to the formatted_documents list
        formatted_documents.append(formatted_document)
    
    ### END CODE HERE ###
    
    # Returns the final augmented prompt string.

    return "\n".join(formatted_documents)

In [35]:
example_data = NEWS_DATA[4:8]

Now let's test your function with some queries.

In [36]:
print(format_relevant_data(example_data))

Title: Prosecutors ask for halt to case against Spain PM's wife, Description: Pedro Sánchez is deciding whether to resign after a case against his wife by an anti-corruption group., Published at: 2024-04-25, URL: https://www.bbc.co.uk/news/world-europe-68895727,
Title: WATCH: Would you pay a tourist fee to enter Venice?, Description: From Thursday visitors making a trip to the famous city at peak times will be charged a trial entrance fee., Published at: 2024-04-25, URL: https://www.bbc.co.uk/news/world-europe-68898441,
Title: Supreme Court divided on whether Trump has immunity, Description: The justices discussed immunity, coups, pardons, Operation Mongoose - and the future of democracy., Published at: 2024-04-25, URL: https://www.bbc.co.uk/news/world-us-canada-68901817,
Title: More than 150 killed as heavy rains pound Tanzania, Description: The prime minister warns that El Niño-triggered heavy rains are likely to continue into May., Published at: 2024-04-25, URL: https://www.bbc.co.u

In [37]:
# Test your function!
unittests.test_format_relevant_data(format_relevant_data)

 All tests passed!


<a id='3-5'></a>
### 3.5 Generate the final prompt

The next function is given to you. It will generate the final prompt, integrating it with the query. Feel free to change the prompt and experiment how different prompts impact the final result!

In [39]:
# EDITABLE CELL

def generate_final_prompt(query, top_k=5, use_rag=True, prompt=None):
    """
    Generates a final prompt based on a user query, optionally incorporating relevant data using retrieval-augmented generation (RAG).

    Args:
        query (str): The user query for which the prompt is to be generated.
        top_k (int, optional): The number of top relevant data pieces to retrieve and incorporate. Default is 5.
        use_rag (bool, optional): A flag indicating whether to use retrieval-augmented generation (RAG)
                                  by including relevant data in the prompt. Default is True.
        prompt (str, optional): A template string for the prompt. It can contain placeholders {query} and {documents}
                                for formatting with the query and formatted relevant data, respectively.

    Returns:
        str: The generated prompt, either consisting solely of the query or expanded with relevant data
             formatted for additional context.
    """
    # If RAG is not being used, format the prompt with just the query or return the query directly
    if not use_rag:
        return query

    # Retrieve the top_k relevant data pieces based on the query
    relevant_data = get_relevant_data(query, top_k=top_k)

    # Format the retrieved relevant data
    retrieve_data_formatted = format_relevant_data(relevant_data)

    # If no custom prompt is provided, use the default prompt template
    if prompt is None:
        prompt = (
            f"Answer the user query below. There will be provided additional information for you to compose your answer. "
            f"The relevant information provided is from 2024 and it should be added as your overall knowledge to answer the query, "
            f"you should not rely only on this information to answer the query, but add it to your overall knowledge."
            f"Query: {query}\n"
            f"2024 News: {retrieve_data_formatted}"
        )
    else:
        # If a custom prompt is provided, format it with the query and formatted relevant data
        prompt = prompt.format(query=query, documents=retrieve_data_formatted)

    return prompt

In [40]:
print(generate_final_prompt("Tell me about the US GDP in the past 3 years."))

Answer the user query below. There will be provided additional information for you to compose your answer. The relevant information provided is from 2024 and it should be added as your overall knowledge to answer the query, you should not rely only on this information to answer the query, but add it to your overall knowledge.Query: Tell me about the US GDP in the past 3 years.
2024 News: Title: America's Economy Is No. 1. That Means Trouble, Description: If you want a single number to capture America’s economic stature, here it is: This year, the U.S. will account for 26.3% of the global gross domestic product, the highest in almost two decades. That’s based on the latest projections from the International Monetary Fund. According to the IMF, Europe’s share of world GDP has dropped 1.4 percentage points since 2018, and Japan’s by 2.1 points. The U.S. share, by contrast, is up 2.3 points., Published at: 2024-04-26, URL: https://www.wsj.com/articles/americas-economy-is-no-1-that-means-tr

<a id='3-6'></a>
### 3.6 LLM call

Now let's integrate the function above to feed an LLM. Its parameters are:

- `query`: the query to be passed to the LLM.
- `use_rag`: a boolean telling whether using RAG or not. This parameter will help you compare queries using a RAG system and not using it.
- `model`: the model to be used. You might change it, but the standard is the Llama 3 Billion parameter.  

In [ ]:
def llm_call(query, top_k = 5, use_rag = True, prompt = None):
    """
    Calls the LLM to generate a response based on a query, optionally using retrieval-augmented generation.

    Args:
        query (str): The user query that will be processed by the language model.
        use_rag (bool, optional): A flag that indicates whether to use retrieval-augmented generation by 
                                  incorporating relevant documents into the prompt. Default is True.

    Returns:
        str: The content of the response generated by the language model.
    """
    

    # Get the prompt with the query + relevant documents
    prompt = generate_final_prompt(query, top_k, use_rag, prompt)

    # Call the LLM
    generated_response = generate_with_single_input(prompt)

    # Get the content
    generated_message = generated_response['content']
    
    return generated_message

In [ ]:
query = "Tell me about the US GDP in the past 3 years."

In [ ]:
print(llm_call(query, use_rag = True))

In [ ]:
print(llm_call(query, use_rag = False))

<a id='4'></a>

<a id='4'></a>
## 4 - Experimenting with your RAG System
---

Now you can experiment with your own queries to see the system in action! You can write any query, and it will display answers both with and without RAG. Keep in mind that the dataset you're working with is related to news data from 2024, so not all queries will be effective in demonstrating the framework. Some example queries you might try include:

* What were the most important events of the past year?
* How is global warming progressing in 2024?
* Tell me about the most recent advances in AI.
* Give me the most important facts from past year.

You can also specify a layout for the augmented prompt that includes placeholders for {query} and {documents} to indicate where they should be inserted within your prompt structure. For example:

```
This is the query: {query}
These are the documents: {documents}
```

In [ ]:
display_widget(llm_call)

Congratulations! You created your first simple RAG system! Keep it up!